In [ ]:
!pip install moviepy pydub openai
!apt-get install ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 5.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.2 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!ls /content/drive/MyDrive/test/yabaidouga.MOV

/content/drive/MyDrive/test/yabaidouga.MOV


In [ ]:
import os

mp4_file_path = '/content/drive/MyDrive/xxx/xxxxx.mp4'
if os.path.exists(mp4_file_path):
    print("File exists")
else:
    print("File does not exist")


File exists


In [ ]:
import os

mp4_file_path = os.path.normpath('/content/drive/MyDrive/test/xxxxx.mp4')
mp3_file_path = os.path.normpath('/content/drive/MyDrive/test/output_audio.mp3')

from moviepy.editor import VideoFileClip

def extract_audio(mp4_file_path, mp3_file_path):
    video = VideoFileClip(mp4_file_path)
    audio = video.audio
    audio.write_audiofile(mp3_file_path)

extract_audio(mp4_file_path, mp3_file_path)



chunk:   3%|▎         | 27/951 [01:23<47:52,  3.11s/it, now=None]

MoviePy - Writing audio in /content/drive/MyDrive/test/output_audio.mp3



chunk:   3%|▎         | 27/951 [01:27<50:00,  3.25s/it, now=None]

MoviePy - Done.


In [ ]:
from pydub import AudioSegment
import math
import os

def split_audio(file_path, size_mb=20):
    audio = AudioSegment.from_file(file_path)
    file_size = os.path.getsize(file_path)
    num_parts = math.ceil(file_size / (size_mb * 1024 * 1024))
    length = len(audio)
    part_length = math.ceil(length / num_parts)
    parts = []

    for i in range(num_parts):
        start = i * part_length
        end = min((i + 1) * part_length, length)
        part = audio[start:end]
        part_file_path = f"/content/drive/MyDrive/part_{i + 1}.mp3"
        part.export(part_file_path, format="mp3")
        parts.append(part_file_path)

    return parts

parts = split_audio('/content/drive/MyDrive/test/output_audio.mp3', 20)

In [ ]:
!pip install --upgrade openai

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
import openai

openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

def transcribe_audio(file_path):
    audio_file = open(file_path, "rb")
    response = openai.audio.transcriptions.create(
        model="whisper-1",
        file=audio_file,
        language="ja"  # 日本語の場合
    )
    return response

def convert_to_text(file_paths):
    with open("/content/drive/MyDrive/final_transcript.txt", "w") as final_text_file:
        for file_path in file_paths:
            transcript = transcribe_audio(file_path)
            final_text_file.write(str(transcript) + "\n")  # 連結する前にtranscriptを文字列に変換

convert_to_text(parts)

In [ ]:
import openai

openai.api_key = "sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx"

def summarize_text(text):
    response = openai.chat.completions.create(
        model="gpt-4o",
        messages=[
            {"role": "system", "content": "あなたはプロの外資系コンサルタントとして、議事録を作成してください。"},
            {"role": "user", "content": text}
        ]
    )
    return response.choices[0].message.content

with open("/content/drive/MyDrive/final_transcript.txt", "r") as file:
    full_text = file.read()

summary_text = summarize_text(full_text)
print(summary_text)


議事録：

1. 発言内容の要約：
   - ある男性が好きな役について話述しています。
   - 彼は劇とそれ以外の生活をきちんと区別している。
   - 元商婦の妻との二人の人間模様が、中国の歴史の変化と一緒に描かれていることを強調している。
   - ストーリーは非常に重く、特に1920年代の役者になる過程での虐待シーンが多く描かれている。

2. 主なポイント：
   - 劇と私生活の区別について
   - 元商婦の妻との関係
   - 中国の歴史との関連性
   - 虐待シーンの頻繁な描写

3. 特記事項：
   - 発言者の感情として、物語が非常に重いと感じていることが強調されています。
   - 具体的な例として、1920年代の役者になる過程での虐待の話があります。

この議事録は、発言内容を簡潔にまとめ、主要なポイントを明確に示すことを目的としています。
